In [ ]:
# Import required libraries
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from IPython.display import display
from tensorflow.keras.models import load_model
from google.colab import drive

# ================================ CONFIGURATION ================================
# Image and sequence settings
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20

# List of classes your model was trained on
CLASSES_LIST = ['fifth', 'first', 'five', 'fourteen']

# Paths
DATASET_PATH = '/content/drive/MyDrive/datasetsign_h264'  # Your original dataset
UPLOAD_PATH = '/content/drive/MyDrive/uploaded_videos'    # Where new videos are uploaded

# ================================ LOAD PRE-TRAINED MODEL ================================
def load_pretrained_model():
    """Load the pre-trained model from Google Drive"""
    # Mount Google Drive
    drive.mount('/content/drive')

    # Model file name (update this with your actual model filename)
    model_file_name = 'convlstm_model___Date_Time_2025_07_16__06_06_09___Loss_0.23585064709186554___Accuracy_0.9411764740943909.h5'
    model_path = os.path.join('/content/drive/MyDrive', model_file_name)

    # Check if model exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at: {model_path}")

    # Load the model
    model = load_model(model_path)
    print(f"✅ Successfully loaded model from: {model_path}")

    return model

# ================================ PREDICTION FUNCTIONS ================================
def extract_frames(video_file_path):
    """
    Extract frames from video file and preprocess them for prediction
    Returns: numpy array of frames or None if extraction fails
    """
    video_reader = cv2.VideoCapture(video_file_path)
    if not video_reader.isOpened():
        print(f"❌ Error: Could not open video {video_file_path}")
        return None

    frames_list = []
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)

    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            print(f"⚠️ Warning: Could not read frame {frame_counter}")
            break

        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255.0
        frames_list.append(normalized_frame)

    video_reader.release()

    if len(frames_list) < SEQUENCE_LENGTH:
        print(f"❌ Error: Only got {len(frames_list)} frames (need {SEQUENCE_LENGTH})")
        return None

    return np.array(frames_list)

def predict_action(model, video_file_path):
    """
    Make prediction on a single video file using the loaded model
    """
    # Extract frames
    frames = extract_frames(video_file_path)
    if frames is None:
        return None

    # Add batch dimension and predict
    frames_batch = np.expand_dims(frames, axis=0)
    predictions = model.predict(frames_batch)[0]

    # Get top prediction
    predicted_class_idx = np.argmax(predictions)
    predicted_class = CLASSES_LIST[predicted_class_idx]
    confidence = predictions[predicted_class_idx]

    return {
        'file_path': video_file_path,
        'predicted_class': predicted_class,
        'confidence': float(confidence),
        'all_predictions': {class_name: float(prob) for class_name, prob in zip(CLASSES_LIST, predictions)}
    }

# ================================ VIDEO DISPLAY FUNCTIONS ================================
def display_video(video_path):
    """Display video in notebook"""
    try:
        clip = VideoFileClip(video_path, audio=False, target_resolution=(300, None))
        display(clip.ipython_display())
    except Exception as e:
        print(f"⚠️ Error displaying video: {str(e)}")

def get_video_files(folder_path):
    """Get list of video files in a folder"""
    if not os.path.exists(folder_path):
        print(f"⚠️ Folder not found: {folder_path}")
        return []

    return sorted([f for f in os.listdir(folder_path)
                   if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))])

# ================================ INTERACTIVE MENU ================================
def interactive_predictor(model):
    """Interactive menu for making predictions"""
    while True:
        print("\n" + "=" * 60)
        print("🎬 VIDEO PREDICTION MENU")
        print("1. Predict on a video from your dataset")
        print("2. Predict on an uploaded video")
        print("3. Exit")

        choice = input("Choose option (1-3): ").strip()

        if choice == '1':
            # Predict on dataset videos
            try:
                folders = [f for f in os.listdir(DATASET_PATH)
                          if os.path.isdir(os.path.join(DATASET_PATH, f))]

                if not folders:
                    print("❌ No class folders found in dataset.")
                    continue

                print("\nAvailable classes:")
                for i, folder in enumerate(folders, 1):
                    print(f"{i}. {folder}")
                print(f"{len(folders)+1}. Back")

                folder_choice = input(f"Select class (1-{len(folders)+1}): ").strip()
                if folder_choice == str(len(folders)+1):
                    continue

                try:
                    selected_folder = folders[int(folder_choice)-1]
                    folder_path = os.path.join(DATASET_PATH, selected_folder)
                    videos = get_video_files(folder_path)

                    if not videos:
                        print(f"❌ No videos found in {selected_folder}")
                        continue

                    print(f"\nVideos in {selected_folder}:")
                    for i, video in enumerate(videos, 1):
                        print(f"{i}. {video}")
                    print(f"{len(videos)+1}. Back")

                    video_choice = input(f"Select video (1-{len(videos)+1}): ").strip()
                    if video_choice == str(len(videos)+1):
                        continue

                    selected_video = videos[int(video_choice)-1]
                    video_path = os.path.join(folder_path, selected_video)

                    # Display and predict
                    print(f"\n▶️ Playing: {selected_video}")
                    display_video(video_path)

                    print("\n🔍 Running prediction...")
                    result = predict_action(model, video_path)

                    if result:
                        print(f"\n🎯 Prediction Result:")
                        print(f"📌 Video: {os.path.basename(result['file_path'])}")
                        print(f"✅ Predicted Class: {result['predicted_class']}")
                        print(f"📈 Confidence: {result['confidence']:.2%}")
                        print("\n📊 All class probabilities:")
                        for class_name, prob in result['all_predictions'].items():
                            print(f"{class_name}: {prob:.2%}")

                except (ValueError, IndexError):
                    print("❌ Invalid selection. Please try again.")

            except FileNotFoundError:
                print(f"❌ Dataset folder not found: {DATASET_PATH}")

        elif choice == '2':
            # Predict on uploaded videos
            try:
                videos = get_video_files(UPLOAD_PATH)

                if not videos:
                    print(f"❌ No videos found in {UPLOAD_PATH}")
                    continue

                print("\nAvailable videos:")
                for i, video in enumerate(videos, 1):
                    print(f"{i}. {video}")
                print(f"{len(videos)+1}. Back")

                video_choice = input(f"Select video (1-{len(videos)+1}): ").strip()
                if video_choice == str(len(videos)+1):
                    continue

                try:
                    selected_video = videos[int(video_choice)-1]
                    video_path = os.path.join(UPLOAD_PATH, selected_video)

                    # Display and predict
                    print(f"\n▶️ Playing: {selected_video}")
                    display_video(video_path)

                    print("\n🔍 Running prediction...")
                    result = predict_action(model, video_path)

                    if result:
                        print(f"\n🎯 Prediction Result:")
                        print(f"📌 Video: {os.path.basename(result['file_path'])}")
                        print(f"✅ Predicted Class: {result['predicted_class']}")
                        print(f"📈 Confidence: {result['confidence']:.2%}")
                        print("\n📊 All class probabilities:")
                        for class_name, prob in result['all_predictions'].items():
                            print(f"{class_name}: {prob:.2%}")

                except (ValueError, IndexError):
                    print("❌ Invalid selection. Please try again.")

            except FileNotFoundError:
                print(f"❌ Upload folder not found: {UPLOAD_PATH}")

        elif choice == '3':
            print("👋 Exiting program.")
            break

        else:
            print("❌ Invalid option. Please choose 1, 2, or 3.")

# ================================ MAIN EXECUTION ================================
if __name__ == "__main__":
    # Load the pre-trained model
    try:
        convlstm_model = load_pretrained_model()
        print("🚀 Model loaded successfully! Ready for predictions.")

        # Start interactive predictor
        interactive_predictor(convlstm_model)

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("Please check:")
        print("- Google Drive is properly mounted")
        print("- Model file exists at the specified path")
        print("- You have correct permissions")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Successfully loaded model from: /content/drive/MyDrive/convlstm_model___Date_Time_2025_07_16__06_06_09___Loss_0.23585064709186554___Accuracy_0.9411764740943909.h5
🚀 Model loaded successfully! Ready for predictions.

🎬 VIDEO PREDICTION MENU
1. Predict on a video from your dataset
2. Predict on an uploaded video
3. Exit
Choose option (1-3): 1

Available classes:
1. fifth
2. first
3. five
4. fourteen
5. Back
Select class (1-5): 3

Videos in five:
1. five.mp4
2. five_10.mp4
3. five_11.mp4
4. five_12.mp4
5. five_14.mp4
6. five_16.mp4
7. five_17.mp4
8. five_18.mp4
9. five_2.mp4
10. five_3.mp4
11. five_4.mp4
12. five_5.mp4
13. five_6.mp4
14. five_7.mp4
15. five_8.mp4
16. five_9.mp4
17. Back
Select video (1-17): 4

▶️ Playing: five_12.mp4
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



t:  96%|█████████▋| 219/227 [00:02<00:00, 78.49it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/datasetsign_h264/five/five_12.mp4, 479700 bytes wanted but 0 bytes read,at frame 226/227, at time 9.83/9.83 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

                                                              

Moviepy - Done !
Moviepy - video ready __temp__.mp4



🔍 Running prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

🎯 Prediction Result:
📌 Video: five_12.mp4
✅ Predicted Class: five
📈 Confidence: 84.04%

📊 All class probabilities:
fifth: 0.02%
first: 2.32%
five: 84.04%
fourteen: 13.62%

🎬 VIDEO PREDICTION MENU
1. Predict on a video from your dataset
2. Predict on an uploaded video
3. Exit
